In [63]:
# import libraries
import numpy as np
import pickle
import os
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
%matplotlib inline
import random
import librosa
from scipy.stats import kurtosis, skew
from sklearn import svm
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score,accuracy_score
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')


In [64]:
# get data
def get_original_feature_label(path):
    # scale both vertical and horizontal between 0 and 1
    scaler = MinMaxScaler()
    feature = []
    label = []
    for i in os.listdir(path):
        a = pd.read_csv(str(path+i),names=["vertical","horizontal"])
        a = np.array(a)
        a = scaler.fit_transform(a)
        #print(str("../data/isolated 2/001/isolated_strokes/"+i))
        f = []
        for j in a[:,0]:
            f.append(j)
        for j in a[:,1]:
            f.append(j)
        feature.append(f)
        if i.endswith("001.csv"):
            label.append(0)
        elif i.endswith("002.csv"):
            label.append(1)
        elif i.endswith("003.csv"):
            label.append(2)
        elif i.endswith("004.csv"):
            label.append(3)
        elif i.endswith("005.csv"):
            label.append(4)
        elif i.endswith("006.csv"):
            label.append(5)
        elif i.endswith("007.csv"):
            label.append(6)
        elif i.endswith("008.csv"):
            label.append(7)
        elif i.endswith("009.csv"):
            label.append(8)
        elif i.endswith("010.csv"):
            label.append(9)
        elif i.endswith("011.csv"):
            label.append(10)
        elif i.endswith("012.csv"):
            label.append(11)
    feature = np.array(feature)
    label = np.array(label)
    
    return feature,label

In [65]:
# train test split 8:2 
def my_train_split(label,feature):
    X_test = []
    X_train = []
    y_train = []
    y_test = []

    for i in range(12):
        id = np.where(label==i)
        tmplabel = label[id]
        tmpfeature = feature[id]
        tmpX_train, tmpX_test, tmpy_train, tmpy_test = train_test_split(tmpfeature, tmplabel,shuffle=True, test_size=0.2, random_state=42)
        for j in range(len(tmpy_train)):
            X_train.append(tmpX_train[j])
            y_train.append(tmpy_train[j])
        for j in range(len(tmpy_test)):
            X_test.append(tmpX_test[j])
            y_test.append(tmpy_test[j])
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    idx = list(range(X_train.shape[0]))
    random.seed(42)
    random.shuffle(idx)
    X_train = X_train[idx]
    y_train = y_train[idx]
    return X_train,X_test,y_train,y_test
    


## User dependent

In [66]:
patient = ["001","002","003","004","005","006"]
output_pck = []
for i in patient:
    feature,label = get_original_feature_label(str("../../data/isolated 2/"+i+"/isolated_strokes/"))
    print(feature.shape)
    # train_test_split
    X_train,X_test,y_train,y_test = my_train_split(label,feature)
    # build model
    clf = svm.SVC(random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(accuracy_score(y_test, y_pred))    
    output = dict()
    output["patient"] = i
    output["X_train"] = X_train
    output["X_test"] = X_test
    output["y_test"] = y_test
    output["y_train"] = y_train
    output["clf"] = clf
    output_pck.append(output)

(120, 2500)
0.9166666666666666
(120, 2500)
0.875
(122, 2500)
0.7307692307692307
(121, 2500)
0.84
(120, 2500)
0.88
(121, 2500)
1.0


In [67]:
print('\nSaving the object')
with open("User-dependent-SVC.pck", "wb") as output_file:
    pickle.dump(output_pck, output_file)


Saving the object


## User independent

In [68]:
patient = ["001","002","003","004","005","006"]
feature = []
label = []
for i in patient:
    f,l = get_original_feature_label(str("../../data/isolated 2/"+i+"/isolated_strokes/"))
    for x in f:
        feature.append(x)         
    for x in l:
        label.append(x)
feature = np.array(feature)
label = np.array(label)
print(feature.shape,label.shape)

(724, 2500) (724,)


In [69]:
# train_test_split
X_train,X_test,y_train,y_test = my_train_split(label,feature)
# build model
clf = svm.SVC(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))  

0.8657718120805369


In [70]:
output = dict()
output["X_train"] = X_train
output["X_test"] = X_test
output["y_test"] = y_test
output["y_train"] = y_train
output["clf"] = clf
print('\nSaving the object')
with open("User-independent-SVC.pck", "wb") as output_file:
    pickle.dump(output, output_file)


Saving the object
